In [1]:
# !pip install streamlit -q
# !pip install pandas numpy plotly -q
# !pip -q install folium
# !pip install streamlit streamlit-folium folium pandas
# !npm install -g localtunnel

In [2]:
# # 나눔고딕 폰트 설치 및 설정
# !apt-get update -qq
# !apt-get install fonts-nanum -qq
# !fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [95]:
%%writefile teamproject.py
import streamlit as st
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from datetime import datetime
import os
import plotly.express as px
import plotly.graph_objects as go
import folium
from folium.plugins import HeatMap
from folium import Popup
from streamlit_folium import st_folium
import numpy as np

# 페이지 설정
st.set_page_config(page_title="SKN_FAMILY_AI_CAMP_16기_1조_팀프로젝트1", layout="wide")

# 폰트 설정 (한 번만 실행)
@st.cache_data
def setup_fonts():
    font_path = r"C:\Users\playdata2\Desktop\플레이데이터\Workspace\Nanum_Gothic\NanumGothic.ttf"
    if os.path.exists(font_path):
        fontprop = fm.FontProperties(fname=font_path, size=10)
        plt.rcParams['font.family'] = 'NanumGothic'
        plt.rcParams['axes.unicode_minus'] = False
    return True

setup_fonts()

# 데이터 로드 함수들 (캐싱 적용)
@st.cache_data
def load_car_data():
    """자동차 등록 정보 로드"""
    df = pd.read_csv('data/CAR_REG_INF.csv')
    df["기준연월_dt"] = pd.to_datetime(df["기준연월"].astype(str), format="%Y%m")
    return df

@st.cache_data
def load_diesel_reg_data():
    """경유차 등록 정보 로드"""
    return pd.read_csv('data/CAR_REG_DSL_INF.csv', encoding='utf-8')

@st.cache_data
def load_diesel_emission_data():
    """경유차 배출량 정보 로드"""
    return pd.read_csv('data/EXHST_AMT_DIESEL.csv', encoding='utf-8')

@st.cache_data
def load_electric_reg_data():
    """전기차 등록 정보 로드"""
    return pd.read_csv("data/CAR_REG_ELEC_INF.csv", dtype={"기준연월": str})

@st.cache_data
def load_charging_station_data():
    """충전소 정보 로드"""
    return pd.read_csv("data/ELEC_CAR_CHRG_STTN.csv", dtype={"기준연월": str})

@st.cache_data
def load_prediction_data(file_path):
    """예측 데이터 로드"""
    if os.path.exists(file_path):
        return pd.read_csv(file_path, encoding='utf-8-sig')
    return None

@st.cache_data
def load_faq_data(file_path):
    """FAQ 데이터 로드"""
    try:
        return pd.read_csv(file_path, encoding='utf-8-sig')
    except FileNotFoundError:
        return None

# 데이터 전처리 함수들
@st.cache_data
def get_month_options():
    """월 옵션 생성"""
    start_point = datetime(2003, 7, 1)
    end_point = datetime(2025, 5, 1)
    months = pd.date_range(start=start_point, end=end_point, freq='MS')
    return [m.strftime('%Y년%m월') for m in months][::-1]

@st.cache_data
def get_extended_month_options():
    """확장된 월 옵션 생성 (2025년 12월까지)"""
    start_point = datetime(2003, 7, 1)
    end_point = datetime(2025, 12, 1)
    months = pd.date_range(start=start_point, end=end_point, freq='MS')
    return [m.strftime('%Y년%m월') for m in months][::-1]

# 세션 상태 관리
def reset_session_for_page(page_name):
    """특정 페이지의 세션 상태만 초기화"""
    keys_to_remove = [key for key in st.session_state.keys() if key.startswith(f"{page_name}_")]
    for key in keys_to_remove:
        del st.session_state[key]

def reset_session_for_tab(tab_name):
    """특정 탭의 세션 상태만 초기화"""
    keys_to_remove = [key for key in st.session_state.keys() if key.startswith(f"{tab_name}_")]
    for key in keys_to_remove:
        del st.session_state[key]

# # 사이드바: 메인 메뉴
# page = st.sidebar.radio("메뉴 선택", ["자동차 등록 현황", "경유차 - 배출량 비교", "전기차 - 충전소 비교", "FAQ"])

# 버튼 형태 (주석 처리 - 원하면 위 코드 대신 사용)
st.sidebar.markdown("### 📋 메뉴 선택")
menu_options = ["자동차 등록 현황", "경유차 - 배출량 비교", "전기차 - 충전소 비교", "FAQ"]

# 세션 상태로 현재 선택된 페이지 관리
if 'selected_page' not in st.session_state:
    st.session_state.selected_page = "자동차 등록 현황"

# 각 메뉴별 버튼 생성
for option in menu_options:
    if st.sidebar.button(
        f"{'🔸' if st.session_state.selected_page == option else '🔹'} {option}",
        key=f"menu_{option}",
        use_container_width=True
    ):
        st.session_state.selected_page = option

page = st.session_state.selected_page

# 페이지 변경 감지 및 세션 초기화
if 'current_page' not in st.session_state:
    st.session_state.current_page = page

if st.session_state.current_page != page:
    reset_session_for_page(st.session_state.current_page)
    st.session_state.current_page = page

# 탭 변경 감지 및 초기화를 위한 함수
def handle_tab_change(page_name, current_tab):
    """탭 변경 감지 및 이전 탭 세션 초기화"""
    tab_tracker_key = f"{page_name}_current_tab"
    
    # 현재 탭 상태 저장
    prev_tab = st.session_state.get(tab_tracker_key, None)
    
    # 탭이 변경된 경우
    if prev_tab != current_tab and prev_tab is not None:
        # 이전 탭의 세션 초기화
        reset_session_for_tab(f"{page_name}_{prev_tab}")
    
    # 현재 탭 상태 업데이트
    st.session_state[tab_tracker_key] = current_tab

# 페이지별 처리
if page == "자동차 등록 현황":
    st.header("🚗 전국 자동차 등록 현황")
    
    # 데이터 로드
    df_car = load_car_data()
    
    sub1, sub2 = st.tabs(["자동차 등록 현황", "전기/경유차 미래 예측"])
    
    with sub1:
        tab_prefix = f"{page}_tab0"
        
        # 수동 초기화 버튼
        if st.button("🔄 이 탭 초기화", key=f"{tab_prefix}_reset"):
            reset_session_for_tab(tab_prefix)
            st.rerun()
        
        st.subheader("필터 선택")
        
        # 필터 옵션
        시도_options = sorted(df_car['시도명'].unique())
        차종_options = sorted(df_car['차종별'].unique())
        연료_options = sorted(df_car['연료별'].unique())
        
        cols = st.columns(3)
        
        # 필터 선택
        selected_시도 = cols[0].multiselect("시도:", 시도_options, default=["전국"], key=f"{tab_prefix}_시도")
        selected_차종 = cols[1].multiselect("차종:", 차종_options, default=["전체"], key=f"{tab_prefix}_차종")
        selected_연료 = cols[2].multiselect("연료:", 연료_options, default=["전체"], key=f"{tab_prefix}_연료")
        
        # 필터 로직 처리
        final_시도 = selected_시도.copy()
        final_차종 = selected_차종.copy()
        final_연료 = selected_연료.copy()
        
        # 전국/전체 값 중복 방지 로직
        if '전국' in final_시도 and len(final_시도) > 1:
            final_시도 = [x for x in final_시도 if x != '전국']
        if '전체' in final_차종 and len(final_차종) > 1:
            final_차종 = [x for x in final_차종 if x != '전체']
        if '전체' in final_연료 and len(final_연료) > 1:
            final_연료 = [x for x in final_연료 if x != '전체']
        
        # 빈 리스트 방지
        if not final_시도:
            final_시도 = ['전국']
        if not final_차종:
            final_차종 = ['전체']
        if not final_연료:
            final_연료 = ['전체']
        
        # 필터 변경사항 알림
        if final_시도 != selected_시도:
            st.info(f"시도 필터에서 '전국'이 자동으로 제거되었습니다: {', '.join(final_시도)}")
        if final_차종 != selected_차종:
            st.info(f"차종 필터에서 '전체'가 자동으로 제거되었습니다: {', '.join(final_차종)}")
        if final_연료 != selected_연료:
            st.info(f"연료 필터에서 '전체'가 자동으로 제거되었습니다: {', '.join(final_연료)}")
        
        # 실제 데이터 필터링에는 final_ 변수들 사용
        selected_시도 = final_시도
        selected_차종 = final_차종
        selected_연료 = final_연료
        
        # 월 옵션
        month_str_list = get_month_options()
        
        # 세션 키
        start_month_key = f'{tab_prefix}_start_month'
        end_month_key = f'{tab_prefix}_end_month'
        
        if start_month_key not in st.session_state:
            st.session_state[start_month_key] = '2024년06월'
        if end_month_key not in st.session_state:
            st.session_state[end_month_key] = '2025년05월'
        
        # 유효한 옵션 계산
        valid_start_options = [m for m in month_str_list if m <= st.session_state[end_month_key]]
        valid_end_options = [m for m in month_str_list if m >= st.session_state[start_month_key]]
        
        # 날짜 선택
        col1, col2 = st.columns(2)
        with col1:
            st.session_state[start_month_key] = st.selectbox(
                "시작 월", valid_start_options,
                index=valid_start_options.index(st.session_state[start_month_key]) if st.session_state[start_month_key] in valid_start_options else 0,
                key=f"{tab_prefix}_start_month_selector"
            )
        
        with col2:
            st.session_state[end_month_key] = st.selectbox(
                "종료 월", valid_end_options,
                index=valid_end_options.index(st.session_state[end_month_key]) if st.session_state[end_month_key] in valid_end_options else len(valid_end_options)-1,
                key=f"{tab_prefix}_end_month_selector"
            )
        
        # 데이터 필터링
        start_date = pd.to_datetime(st.session_state[start_month_key], format='%Y년%m월')
        end_date = pd.to_datetime(st.session_state[end_month_key], format='%Y년%m월') + pd.offsets.MonthEnd(0)
        
        filtered_df = df_car[
            (df_car["시도명"].isin(selected_시도)) & 
            (df_car["차종별"].isin(selected_차종)) & 
            (df_car["연료별"].isin(selected_연료)) &
            (df_car["기준연월_dt"] >= start_date) &
            (df_car["기준연월_dt"] <= end_date) &
            (df_car["차량대수"] > 0)
        ]
        
        # 그래프 출력
        if not filtered_df.empty:
            df_line = filtered_df.groupby("기준연월_dt")["차량대수"].sum().reset_index()
            
            전체_선택 = ('전국' in selected_시도) or ('전체' in selected_차종) or ('전체' in selected_연료)
            
            # df_line["차량대수"] = df_line["차량대수"] / 10000
            y_label = "등록대수"
            
            df_line["기준연월_dt"] = df_line["기준연월_dt"].dt.strftime('%Y년 %m월')
            
            # 필터 정보 문자열 생성 (각 필터별로 / 구분, 필터 내에서는 , 구분)
            시도_str = ', '.join(selected_시도)
            차종_str = ', '.join(selected_차종)
            연료_str = ', '.join(selected_연료)
            filter_info = f"{시도_str} / {차종_str} / {연료_str}"
            
            st.subheader(f"📊 월별 자동차 등록대수 현황({filter_info})")

            # === 여기에 Y축 눈금 설정 코드 추가! ===
            max_val = df_line["차량대수"].max()
            min_val = df_line["차량대수"].min()
            tick_interval = max(10000, int((max_val - min_val) / 8 / 10000) * 10000)
            
            fig = px.bar(
                df_line, x="기준연월_dt", y="차량대수",
                labels={"기준연월_dt": "월", "차량대수": y_label},
                title="월별 자동차 등록대수",
                color_discrete_sequence=["skyblue"]
            )

            # Y축 눈금값은 실제 값 (예: 0, 50000, 100000)
            tick_vals = np.arange(0, max_val + tick_interval, tick_interval)

            # Y축 표시는 만 대 단위 (예: 0, 5, 10)
            tick_texts = [f"{int(val/10000)}" for val in tick_vals]
            
            fig.update_traces(hovertemplate='<b>%{x}</b><br>등록대수: %{y:,} 대<extra></extra>')
            fig.update_layout(
                height=500,
                title=dict(text="월별 자동차 등록대수", x=0.5, xanchor='center'),
                yaxis=dict(rangemode='tozero')
            )
            
            st.plotly_chart(fig, use_container_width=True)
        else:
            st.warning("선택한 조건에 해당하는 데이터가 없습니다.")
    
    with sub2:
        tab_prefix = f"{page}_tab1"
        
        # 수동 초기화 버튼
        if st.button("🔄 이 탭 초기화", key=f"{tab_prefix}_reset"):
            reset_session_for_tab(tab_prefix)
            st.rerun()
        
        st.subheader("미래 예측 모델 결과")
        
        ev_tab, diesel_tab = st.tabs(["전기차 미래 예측", "경유차 미래 예측"])
        
        with ev_tab:
            subtab_prefix = f"{tab_prefix}_subtab0"
            
            # 서브탭 수동 초기화 버튼
            if st.button("🔄 이 서브탭 초기화", key=f"{subtab_prefix}_reset"):
                reset_session_for_tab(subtab_prefix)
                st.rerun()
            
            st.markdown("### 전기차 등록대수 예측")
            
            # 월 옵션
            month_options = get_extended_month_options()
            
            # 세션 키
            start_month_key = f'{subtab_prefix}_start_month'
            end_month_key = f'{subtab_prefix}_end_month'
            
            if start_month_key not in st.session_state:
                st.session_state[start_month_key] = '2025년01월'
            if end_month_key not in st.session_state:
                st.session_state[end_month_key] = '2025년12월'
            
            # 유효한 옵션 계산
            valid_start_options = [m for m in month_options if m <= st.session_state[end_month_key]]
            valid_end_options = [m for m in month_options if m >= st.session_state[start_month_key]]
            
            # 기간 선택
            col1, col2 = st.columns(2)
            with col1:
                st.session_state[start_month_key] = st.selectbox(
                    "시작 월", valid_start_options,
                    index=valid_start_options.index(st.session_state[start_month_key]) if st.session_state[start_month_key] in valid_start_options else 0,
                    key=f"{subtab_prefix}_start_month_selector"
                )
            
            with col2:
                st.session_state[end_month_key] = st.selectbox(
                    "종료 월", valid_end_options,
                    index=valid_end_options.index(st.session_state[end_month_key]) if st.session_state[end_month_key] in valid_end_options else len(valid_end_options)-1,
                    key=f"{subtab_prefix}_end_month_selector"
                )
            
            # 선택된 기간을 datetime으로 변환
            start_date = pd.to_datetime(st.session_state[start_month_key], format='%Y년%m월')
            end_date = pd.to_datetime(st.session_state[end_month_key], format='%Y년%m월') + pd.offsets.MonthEnd(0)
            
            # 예측 데이터 로드
            df_ev = load_prediction_data('data/전기차_누적등록.csv')
            
            if df_ev is not None and '기준연월' in df_ev.columns:
                df_ev['기준연월'] = pd.to_datetime(df_ev['기준연월'])
                
                # 선택된 기간으로 필터링
                mask = (df_ev['기준연월'] >= start_date) & (df_ev['기준연월'] <= end_date)
                df_plot = df_ev.loc[mask].copy()
                
                if not df_plot.empty:
                    df_plot['연월'] = df_plot['기준연월'].dt.strftime('%Y년%m월')
                    
                    fig = go.Figure()
                    fig.add_trace(go.Scatter(
                        x=df_plot['연월'], y=df_plot['차량대수'],
                        mode='lines+markers',
                        line=dict(color='royalblue', width=3),
                        marker=dict(size=6),
                        name="차량대수",
                        hovertemplate='<b>%{x}</b><br>등록대수: %{y:,} 대<extra></extra>'
                    ))
                    
                    fig.update_layout(
                        title=dict(text="📈 전기차 등록대수 추이", x=0.5, xanchor='center'),
                        xaxis_title="월", yaxis_title="등록대수",
                        height=600,
                        yaxis=dict(range=[df_plot['차량대수'].min() * 0.9, df_plot['차량대수'].max() * 1.1])
                    )
                    
                    st.plotly_chart(fig, use_container_width=True)
                else:
                    st.warning("선택한 기간에 해당하는 데이터가 없습니다.")
            else:
                st.warning("전기차 예측 데이터를 찾을 수 없습니다.")
        
        with diesel_tab:
            subtab_prefix = f"{tab_prefix}_subtab1"
            
            # 서브탭 수동 초기화 버튼
            if st.button("🔄 이 서브탭 초기화", key=f"{subtab_prefix}_reset"):
                reset_session_for_tab(subtab_prefix)
                st.rerun()
            
            st.markdown("### 경유차 등록대수 예측")
            
            # 월 옵션
            extended_month_list = get_extended_month_options()
            
            # 세션 키
            start_month_key = f'{subtab_prefix}_start_month'
            end_month_key = f'{subtab_prefix}_end_month'
            
            if start_month_key not in st.session_state:
                st.session_state[start_month_key] = '2025년01월'
            if end_month_key not in st.session_state:
                st.session_state[end_month_key] = '2025년12월'
            
            # 유효한 옵션 계산
            valid_start_options = [m for m in extended_month_list if m <= st.session_state[end_month_key]]
            valid_end_options = [m for m in extended_month_list if m >= st.session_state[start_month_key]]
            
            # 기간 선택
            col1, col2 = st.columns(2)
            with col1:
                st.session_state[start_month_key] = st.selectbox(
                    "시작 월", valid_start_options,
                    index=valid_start_options.index(st.session_state[start_month_key]) if st.session_state[start_month_key] in valid_start_options else 0,
                    key=f"{subtab_prefix}_start_month_selector"
                )
            
            with col2:
                st.session_state[end_month_key] = st.selectbox(
                    "종료 월", valid_end_options,
                    index=valid_end_options.index(st.session_state[end_month_key]) if st.session_state[end_month_key] in valid_end_options else len(valid_end_options)-1,
                    key=f"{subtab_prefix}_end_month_selector"
                )
            
            # 선택된 기간을 datetime으로 변환
            start_date = pd.to_datetime(st.session_state[start_month_key], format='%Y년%m월')
            end_date = pd.to_datetime(st.session_state[end_month_key], format='%Y년%m월') + pd.offsets.MonthEnd(0)
            
            # 예측 데이터 로드
            df_pred = load_prediction_data('data/경유차_누적등록.csv')
            
            if df_pred is not None and '기간' in df_pred.columns:
                df_pred['기간'] = pd.to_datetime(df_pred['기간'])
                
                # 선택된 기간으로 필터링
                mask = (df_pred['기간'] >= start_date) & (df_pred['기간'] <= end_date)
                df_plot = df_pred.loc[mask].copy()
                
                if not df_plot.empty:
                    df_plot['연월'] = df_plot['기간'].dt.strftime('%Y년 %m월')
                    # df_plot['차량대수'] = df_plot['차량대수'] / 1000
                    
                    fig = px.line(
                        df_plot, x='연월', y='차량대수',
                        labels={'연월': '월', '차량대수': '등록대수'},
                        title="📊 경유차 등록대수 추이",
                        color_discrete_sequence=['skyblue']
                    )
                    
                    fig.update_traces(hovertemplate='<b>%{x}</b><br>등록대수: %{y:,}천 대<extra></extra>')
                    fig.update_layout(
                        height=600,
                        title=dict(text="📊 경유차 등록대수 추이", x=0.5, xanchor='center'),
                        # xaxis_tickangle=35,
                        yaxis=dict(range=[df_plot['차량대수'].min() * 0.9, df_plot['차량대수'].max() * 1.1])
                    )
                    
                    st.plotly_chart(fig, use_container_width=True)
                else:
                    st.warning("선택한 기간에 해당하는 데이터가 없습니다.")
            else:
                st.warning("경유차 예측 데이터를 찾을 수 없습니다.")

elif page == "경유차 - 배출량 비교":
    st.header("📈 경유차 등록대수, 배출가스 배출량 비교")
    
    # 데이터 로드
    CAR_REG_DSL = load_diesel_reg_data()
    EXHST_AMT_DISEL = load_diesel_emission_data()
    
    st.subheader("기준연도")
    
    page_prefix = f"{page}"
    months = sorted(CAR_REG_DSL["기준연도"].unique())
    default = months[-1]
    
    selected_year = st.select_slider(
        "", options=months, value=default,
        key=f"{page_prefix}_year_slider"
    )
    
    # 데이터 필터링
    filtered_df = CAR_REG_DSL[CAR_REG_DSL['기준연도'] == selected_year]
    filtered_df2 = EXHST_AMT_DISEL[EXHST_AMT_DISEL['기준연도'] == selected_year]
    
    # 물질 선택
    substances = ["일산화탄소", "질소산화물", "휘발성유기화합물", "미세먼지", "초미세먼지"]
    total_cars = filtered_df['차량대수'].sum()
    
    cols = st.columns(2)
    with cols[0]:
        st.subheader(f"전국 경유차 등록대수({selected_year}년): {total_cars:,}대")
    
    selected_substance = cols[1].selectbox(
        "배출가스:", substances,
        key=f"{page_prefix}_substance_select"
    )
    
    # 지도 생성
    left_col, right_col = st.columns(2)
    center = [36.8124, 127.7888]
    
    # 왼쪽 히트맵 (등록대수)
    with left_col:
        heat_data_1 = filtered_df[['위도', '경도', '차량대수']].values.tolist()
        
        m = folium.Map(location=center, zoom_start=7)
        if heat_data_1:
            HeatMap(
                heat_data_1, radius=30, blur=15, max_zoom=13,
                max_intensity=int(CAR_REG_DSL['차량대수'].max())
            ).add_to(m)
        
        # 마커 추가
        for i in filtered_df.index:
            folium.Marker(
                location=[filtered_df.loc[i, '위도'], filtered_df.loc[i, '경도']],
                tooltip=filtered_df.loc[i, '시도명'],
                popup=Popup(f"<b>등록대수: {int(filtered_df.loc[i, '차량대수']):,}</b>", max_width=300),
                icon=folium.Icon(color='darkblue', icon='car', prefix='fa')
            ).add_to(m)
        
        st.subheader(f"🚗 시도별 경유차 등록대수({selected_year}년)")
        st_folium(m, height=500, key=f"{page_prefix}_map1")
    
    # 오른쪽 히트맵 (배출량)
    with right_col:
        heat_data_2 = filtered_df2[['위도', '경도', selected_substance]].values.tolist()
        
        m2 = folium.Map(location=center, zoom_start=7)
        if heat_data_2:
            HeatMap(
                heat_data_2, radius=30, blur=15, max_zoom=13,
                max_intensity=int(EXHST_AMT_DISEL[selected_substance].max())
            ).add_to(m2)
        
        # 마커 추가
        for i in filtered_df2.index:
            value = filtered_df2.loc[i, selected_substance]
            folium.Marker(
                location=[filtered_df2.loc[i, '위도'], filtered_df2.loc[i, '경도']],
                tooltip=filtered_df2.loc[i, '시도명'],
                popup=Popup(f"<b>{selected_substance} 배출량: {int(value):,}</b>", max_width=300),
                icon=folium.Icon(color='lightgray', icon='cloud', prefix='fa')
            ).add_to(m2)
        
        st.subheader(f"💨 시도별 {selected_substance} 배출량({selected_year}년)")
        st_folium(m2, height=500, key=f"{page_prefix}_map2")

elif page == "전기차 - 충전소 비교":
    st.header("📈 전기차 등록대수, 전기차 충전기 대수 비교")
    
    # 시도 중심 좌표
    sido_centers = {
        "강원": [37.69442222, 127.8908417],
        "경기": [37.29535833, 127.6396222],
        "경상": [35.86952722, 128.6061745],
        "서울": [37.53609444, 126.9675222],
        "인천": [37.44971062, 126.7309669],
        "전라": [35.318125, 126.9901639],
        "제주": [33.25235, 126.5125556],
        "충청": [36.804125, 127.1524667]
    }
    
    # 데이터 로드
    reg_df = load_electric_reg_data()
    sttn_df = load_charging_station_data()
    
    page_prefix = f"{page}"
    months = sorted(reg_df["기준연월"].unique())
    default = months[-1]
    
    st.subheader("기준연도")
    기준연월 = st.select_slider(
        "", options=months, value=default,
        key=f"{page_prefix}_month_slider"
    )
    
    # 데이터 필터링
    reg_flt = reg_df[reg_df["기준연월"] == 기준연월]
    sttn_flt = sttn_df[sttn_df["기준연월"] == 기준연월]
    
    reg_flt["차량대수"] = pd.to_numeric(reg_flt["차량대수"], errors="coerce")
    sttn_flt["충전기대수"] = pd.to_numeric(sttn_flt["충전기대수"], errors="coerce")
    
    # 시도별 집계
    reg_by_sido = reg_flt.groupby("시도명")["차량대수"].sum()
    sttn_by_sido = sttn_flt.groupby("시도명")["충전기대수"].sum()
    
    center_korea = [36.8124, 127.7888]
    
    # 지도 생성
    col1, col2 = st.columns(2)
    
    # 전기차 등록 지도
    with col1:
        reg_map = folium.Map(location=center_korea, zoom_start=7)
        
        HeatMap(
            reg_flt[["위도", "경도", "차량대수"]].values.tolist(),
            radius=30, blur=15, max_zoom=13, min_opacity=0.2,
            max_intensity=int(reg_df['차량대수'].max())
        ).add_to(reg_map)
        
        # 마커 추가
        for sido, latlon in sido_centers.items():
            if sido in reg_by_sido.index:
                folium.Marker(
                    location=latlon, tooltip=sido,
                    popup=folium.Popup(f"<b>등록대수:</b> {int(reg_by_sido[sido]):,}", max_width=200),
                    icon=folium.Icon(color="blue", icon="car", prefix="fa")
                ).add_to(reg_map)
        
        st.subheader(f"🚙 시도별 전기차 등록대수({기준연월[:4]}년 {기준연월[-2:]}월)")
        st_folium(reg_map, key=f"reg_{page_prefix}_{기준연월}", height=500)
    
    # 충전소 지도
    with col2:
        sttn_map = folium.Map(location=center_korea, zoom_start=7)
        
        HeatMap(
            sttn_flt[["위도", "경도", "충전기대수"]].values.tolist(),
            radius=30, blur=15, max_zoom=13, min_opacity=0.2,
            max_intensity=int(sttn_df['충전기대수'].max())
        ).add_to(sttn_map)
        
        # 마커 추가
        for sido, latlon in sido_centers.items():
            if sido in sttn_by_sido.index:
                folium.Marker(
                    location=latlon, tooltip=sido,
                    popup=folium.Popup(f"<b>충전기대수:</b> {int(sttn_by_sido[sido]):,}", max_width=200),
                    icon=folium.Icon(color="green", icon="bolt", prefix="fa")
                ).add_to(sttn_map)
        
        st.subheader(f"🔌 시도별 충전기 대수({기준연월[:4]}년 {기준연월[-2:]}월)")
        st_folium(sttn_map, key=f"sttn_{page_prefix}_{기준연월}", height=500)

else:  # FAQ
    st.header("❓ 기업별 FAQ 조회 시스템")
    
    # FAQ 파일 정보
    company_files = {
        "제네시스": "data/genesis_faq.csv",
        "현대": "data/hyundai_faq.csv",
        "기아": "data/kia_faq.csv",
        "KGM모빌리티": "data/kgm_faq.csv",
        "지엠(GM)": "data/gm_faq.csv"
    }
    
    topic_keywords = {
        "제네시스": ['전체', '차량구매', '정비예약', '홈페이지', 'MY GENESIS 앱', '인카페이먼트', '커넥티드', '빌트인 캠', 'OTA'],
        "현대": ['전체', '차량구매', '차량정비', '홈페이지', '블루멤버스', '모젠서비스', '블루링크', '디지털 키', '빌트인캠'],
        "기아": ['전체', '차량구매', '차량정비', '기아멤버스', '홈페이지', 'PBV', '기타'],
        "KGM모빌리티": ['전체', '홈페이지', '구매/영업', '세제/법규', '차량정비', '부품'],
        "지엠(GM)": ['전체', '온스타', '앱', '원격제어', '차량진단']
    }
    
    page_prefix = f"{page}"
    company = st.selectbox("기업 선택", list(company_files.keys()), key=f"{page_prefix}_company")
    
    # FAQ 데이터 로드
    df_faq = load_faq_data(company_files[company])
    
    if df_faq is not None:
        st.success(f"{company} FAQ ({len(df_faq)}건) 불러오기 완료")
        
        # 분류 키워드 선택
        topics = topic_keywords[company]
        selected_topic = st.selectbox("분류 키워드 선택", topics, key=f"{page_prefix}_topic")
        
        # 검색 필터링
        if selected_topic != '전체':
            df_faq = df_faq[df_faq['분류'].str.contains(selected_topic, na=False)]
        
        keyword = st.text_input("🔍 질문 검색", key=f"{page_prefix}_keyword")
        if keyword:
            df_faq = df_faq[df_faq['질문'].str.contains(keyword, case=False, na=False)]
        
        # FAQ 출력
        for _, row in df_faq.iterrows():
            with st.expander(row['질문']):
                st.markdown(f"**[분류]** {row['분류']}")
                st.markdown(f"**[답변]** {row['답변']}")
    else:
        st.error(f"{company_files[company]} 파일을 찾을 수 없습니다.")

Overwriting teamproject.py


In [78]:
# !wget -q -O - ipv4.icanhazip.com

In [79]:
!streamlit run teamproject.py & npx localtunnel --port 8501

^C
